# predict on sequence

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [ ]:
import simushape_nostruct  as sn
import simushape as ss
import numpy as np
# the plan is to write my predictions in a react file
if True:
    data = ss.get_all_data('data/RNA16.react','data/RNA16.dbn')
    #data2 = ss.get_all_data('data/RNA20.react','data/RNA20.dbn')
    data.pop("GLYCFN",None)
    predict_keys= data.keys()
    #data.update(data2)
    res=[]
    for e in predict_keys: 
        train = data.keys()
        train.remove(e)
        model = sn.make_model(data,train)
        target_sequence = data[e][1]
        my_react = np.array(sn.predict(model,target_sequence))
        res.append(">%s"%e)
        res.append('\n'.join(["%s\t%.4f" % (i,e) for i,e in enumerate(my_react)]))
        res.append('')
        print 'x'

    with open("data/predictions/shaker_prediction_eznostr.react",'w') as f: f.write('\n'.join(res))


data for 'tRNA-asp' is corrupted, ignoring...


# DRAW FUNCTION FOR THE SHAPE DATA

In [ ]:
from graphlearn01.utils import draw
import numpy as np
from scipy.stats import pearsonr as pear
from scipy.stats import spearmanr as spear
import simushape  as ss


def annotate(g,shap):
    n=g.nodes()
    n.sort()
    for e,i in zip(n,shap):
        g.node[e]["importance"]= [i]
        g.node[e]["none"]= ''
    return g

def draw3(graph, shape_list):
    graphs=[annotate(graph.copy(),shape) for shape in shape_list  ]
    draw.graphlearn(graphs,size=15,layout="RNA",vertex_label='none',edge_alpha=0.05,vertex_size=150,vertex_border=False, scoretricks=True)
    

def mycorr(train, prediction):
    train = np.array(train)
    prediction = np.array(prediction)
    predict2, train2 = ss.mask(prediction,train)
    p =  pear(train2,predict2)[0]
    s =  spear(train2,predict2)[0]
    return "pearson: %.3f spearman %.3f variances: %.3f %.3f" % (p,s, train2.var(), predict2.var())


klmer2 = ss.read_react("data/predictions/RNA16_hlmer_prediction.react")
#klmer3 = ss.read_react("data/hlmer_3_prediction.react")
shaker = ss.read_react("data/predictions/shaker_prediction_eznostr.react")
sukust = ss.read_react("data/predictions/RNA16_sukosd_prediction.react")
RNA16 =  ss.read_react("data/RNA16.react")
allreacts = [RNA16, shaker,klmer2]
RNA16_complete = ss.get_all_data('data/RNA16.react','data/RNA16.dbn')# load includes sanity check;; name -> react,sequence,dotbracket


for e in RNA16_complete:
    print "rna name", e
    graph = ss.eden_rna.sequence_dotbracket_to_graph(RNA16_complete[e][1],RNA16_complete[e][2])
    graph.graph["structure"] = RNA16_complete[e][2]
    draw3(graph,[ react[e] for react in allreacts ])
    print "shaler:", mycorr(RNA16[e],shaker[e])
    print "hlmer:",mycorr(RNA16[e],klmer2[e])
    #print "sukosd:",mycorr(RNA16[e],sukust[e])

In [ ]:

from graphlearn01.utils import draw
import numpy as np
from scipy.stats import pearsonr as pear
from scipy.stats import spearmanr as spear
import simushape  as ss


RNA20 =  ss.read_react("data/RNA20.react")
allreacts = [RNA20]
RNA20_complete = ss.get_all_data('data/RNA20.react','data/RNA20.dbn')# load includes sanity check;; name -> react,sequence,dotbracket


def draw_deflayout(graph, shape_list):
    graphs=[annotate(graph.copy(),shape) for shape in shape_list  ]
    draw.graphlearn(graphs,size=15,layout="RNA",vertex_label='none',edge_alpha=0.05,vertex_size=150,vertex_border=False, scoretricks=True)
    

for e in RNA20_complete:
    print "rna name", e
    graph = ss.eden_rna.sequence_dotbracket_to_graph(RNA20_complete[e][1],RNA20_complete[e][2])
    graph.graph["structure"] = RNA20_complete[e][2]
    draw_deflayout(graph,[ RNA20[e] ])
  
  